In [1]:
#| default_exp data.transforms

In [2]:
#| hide
!pip install -q nbdev

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| hide
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/rank-bert

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/rank-bert


In [5]:
#| export
import torch
from fastai.text.all import *
from transformers import AutoTokenizer
from torch.utils.data._utils.collate import default_collate


## Batch transformation utilities for BERT rank experiments.

This module provides the necessary transform classes for tokenizing batches of text
for BERT models, handling both single-text and text-pair inputs.


In [6]:
#| export
class TransTensorText(TensorBase): pass

class Undict(Transform):
    """Transform to convert tokenizer output dict to TransTensorText for decoding"""
    def decodes(self, x:dict):
        if 'input_ids' in x:
            res = TransTensorText(x['input_ids'])
            return res
        return x

class TokBatchTransform(Transform):
    """
    Tokenizes texts in batches using pretrained HuggingFace tokenizer.
    Following the reference implementation pattern.
    """
    def __init__(self, pretrained_model_name=None, tokenizer_cls=AutoTokenizer,
                 config=None, tokenizer=None, with_labels=False,
                 padding=True, truncation=True, max_length=None, **kwargs):
        if tokenizer is None:
            tokenizer = tokenizer_cls.from_pretrained(pretrained_model_name, config=config)
        self.tokenizer = tokenizer
        self.kwargs = kwargs
        self._two_texts = False
        store_attr()

    def encodes(self, batch):
        # Handle initialization case - if batch is None or empty
        if batch is None or len(batch) == 0:
            # Return a dummy structure for initialization
            dummy = {'input_ids': torch.zeros(1, 10, dtype=torch.long),
                    'attention_mask': torch.zeros(1, 10, dtype=torch.long)}

            if 'token_type_ids' in self.tokenizer.model_input_names:
                dummy['token_type_ids'] = torch.zeros(1, 10, dtype=torch.long)

            return (dummy, torch.zeros(1, dtype=torch.long))

        # Check if first item is a tuple (text pair)
        if is_listy(batch[0][0]):
            self._two_texts = True
            texts = ([s[0][0] for s in batch], [s[0][1] for s in batch])
        else:
            texts = ([s[0] for s in batch],)

        # Tokenize texts
        inputs = self.tokenizer(*texts,
                              add_special_tokens=True,
                              padding=self.padding,
                              truncation=self.truncation,
                              max_length=self.max_length,
                              return_tensors='pt',
                              **self.kwargs)

        # Collate labels
        labels = default_collate([s[1:] for s in batch])

        # Return structure depends on with_labels flag
        if self.with_labels:
            inputs['labels'] = labels[0]
            return (inputs, )
        else:
            return (inputs, ) + tuple(labels)

    def decodes(self, x):
        if isinstance(x, TransTensorText):
            if self._two_texts:
                x1, x2 = split_by_sep(x, self.tokenizer.sep_token_id)
                return (TitledStr(self.tokenizer.decode(x1.cpu(), skip_special_tokens=True)),
                        TitledStr(self.tokenizer.decode(x2.cpu(), skip_special_tokens=True)))
            return TitledStr(self.tokenizer.decode(x.cpu(), skip_special_tokens=True))
        return x

In [7]:
import nbdev; nbdev.nbdev_export()